In [1]:
## mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install snntorch

In [3]:
## os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"


In [4]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import cv2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from pathlib import Path
from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
from sklearn.metrics import matthews_corrcoef
import gc

# --- SNN Imports ---
import snntorch as snn
from snntorch import surrogate
from snntorch import utils

# ==========================================
# 1. CONFIGURATION (STABLE MODE)
# ==========================================
CONFIG = {
    "base_dir": "/content/drive/MyDrive/GlacierHack_practice/Train",
    "project_dir": "/content/drive/MyDrive/Glacier_SNN_Project",
    "model_type": "SNN",
    "time_steps": 4,
    "beta": 0.9,
    "epochs": 30,
    "batch_size": 2,       # Safe for Float32
    "lr": 1e-3,
    "num_workers": 2,
    "seed": 42,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu")
}

torch.cuda.empty_cache()
gc.collect()
os.makedirs(CONFIG['project_dir'], exist_ok=True)

def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(CONFIG['seed'])

# ==========================================
# 2. DATASET
# ==========================================
class GlacierDataset(Dataset):
    def __init__(self, base_dir, transform=None):
        self.base_dir = Path(base_dir)
        self.band_dirs = [self.base_dir / f"Band{i}" for i in range(1, 6)]
        self.label_dir = self.base_dir / "labels"
        self.ids = sorted([p.stem for p in self.band_dirs[0].glob("*.tif")])
        self.transform = transform

    def __len__(self): return len(self.ids)

    def __getitem__(self, idx):
        img_id = self.ids[idx]
        bands = [cv2.imread(str(d / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED).astype(np.float32) for d in self.band_dirs]
        image = np.stack(bands, axis=-1)
        label = cv2.imread(str(self.label_dir / f"{img_id}.tif"), cv2.IMREAD_UNCHANGED)
        if label.ndim == 3: label = cv2.cvtColor(label, cv2.COLOR_BGR2GRAY)
        mask = np.zeros_like(label, dtype=np.uint8)
        mask[label == 85] = 1; mask[label == 170] = 2; mask[label == 255] = 3

        p02, p98 = np.percentile(image, 2), np.percentile(image, 98)
        image = np.clip(image, p02, p98)
        image = (image - image.min()) / (image.max() - image.min() + 1e-6)

        if self.transform:
            aug = self.transform(image=image, mask=mask)
            return aug["image"].float(), aug["mask"].long()
        return torch.tensor(image.transpose(2,0,1)).float(), torch.tensor(mask).long()

class Wrapper(Dataset):
    def __init__(self, ds, t): self.ds, self.t = ds, t
    def __len__(self): return len(self.ds)
    def __getitem__(self, i):
        img, mask = self.ds[i]
        img = img.numpy().transpose(1,2,0); mask = mask.numpy()
        res = self.t(image=img, mask=mask)
        return res['image'], res['mask'].long()

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5), A.VerticalFlip(p=0.5), A.RandomRotate90(p=0.5),
    A.GridDistortion(p=0.3),
    ToTensorV2(),
])
val_transform = A.Compose([ToTensorV2()])

full_dataset = GlacierDataset(CONFIG['base_dir'], transform=None)
val_len = int(len(full_dataset)*0.2)
train_ds, val_ds = random_split(full_dataset, [len(full_dataset)-val_len, val_len])

train_loader = DataLoader(Wrapper(train_ds, train_transform), batch_size=CONFIG['batch_size'], shuffle=True, num_workers=2)
val_loader = DataLoader(Wrapper(val_ds, val_transform), batch_size=CONFIG['batch_size'], shuffle=False, num_workers=2)

# ==========================================
# 3. SNN ARCHITECTURE
# ==========================================
class SNNBlock(nn.Module):
    def __init__(self, in_c, out_c):
        super().__init__()
        spike_grad = surrogate.fast_sigmoid()
        self.net = nn.Sequential(
            nn.Conv2d(in_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            snn.Leaky(beta=CONFIG['beta'], spike_grad=spike_grad, init_hidden=True),

            nn.Conv2d(out_c, out_c, 3, padding=1, bias=False),
            nn.BatchNorm2d(out_c),
            snn.Leaky(beta=CONFIG['beta'], spike_grad=spike_grad, init_hidden=True)
        )
    def forward(self, x): return self.net(x)

class ProjectUNet(nn.Module):
    def __init__(self, in_ch=5, n_classes=4):
        super().__init__()
        Block = SNNBlock
        self.inc = Block(in_ch, 32)
        self.down1 = nn.Sequential(nn.MaxPool2d(2), Block(32, 64))
        self.down2 = nn.Sequential(nn.MaxPool2d(2), Block(64, 128))
        self.down3 = nn.Sequential(nn.MaxPool2d(2), Block(128, 256))

        self.up1 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.conv1 = Block(256, 128)
        self.up2 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.conv2 = Block(128, 64)
        self.up3 = nn.ConvTranspose2d(64, 32, kernel_size=2, stride=2)
        self.conv3 = Block(64, 32)
        self.outc = nn.Conv2d(32, n_classes, 1)

    def forward(self, x):
        # MANUAL RESET inside forward is dangerous for BPTT
        # We handle reset in the training loop instead

        spk_rec = []
        for step in range(CONFIG['time_steps']):
            x1 = self.inc(x)
            x2 = self.down1(x1)
            x3 = self.down2(x2)
            x4 = self.down3(x3)

            x_up1 = self.up1(x4)
            if x_up1.shape != x3.shape: x_up1 = F.interpolate(x_up1, size=x3.shape[2:])
            x5 = self.conv1(torch.cat([x3, x_up1], dim=1))

            x_up2 = self.up2(x5)
            if x_up2.shape != x2.shape: x_up2 = F.interpolate(x_up2, size=x2.shape[2:])
            x6 = self.conv2(torch.cat([x2, x_up2], dim=1))

            x_up3 = self.up3(x6)
            if x_up3.shape != x1.shape: x_up3 = F.interpolate(x_up3, size=x1.shape[2:])
            x7 = self.conv3(torch.cat([x1, x_up3], dim=1))

            out = self.outc(x7)
            spk_rec.append(out)

        return torch.stack(spk_rec).mean(0)

# ==========================================
# 4. VISUALIZATION
# ==========================================
def save_plot_and_sample(history, sample_data, epoch):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history['loss'], label='Train Loss')
    plt.title("SNN Loss")
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history['mcc'], label='Val MCC', color='green')
    plt.title("SNN MCC")
    plt.legend()
    plt.savefig(f"{CONFIG['project_dir']}/SNN_metrics.png")
    plt.close()

    img, gt, pred = sample_data
    rgb = img[[3,2,1]].transpose(1,2,0)
    rgb = (rgb - rgb.min()) / (rgb.max() - rgb.min() + 1e-6)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1); plt.imshow(rgb); plt.title("Input")
    plt.subplot(1, 3, 2); plt.imshow(gt, cmap='nipy_spectral'); plt.title("Ground Truth")
    plt.subplot(1, 3, 3); plt.imshow(pred, cmap='nipy_spectral'); plt.title(f"Pred (Ep {epoch})")
    plt.savefig(f"{CONFIG['project_dir']}/SNN_sample.png")
    plt.close()

# ==========================================
# 5. TRAINING
# ==========================================
def manual_reset(model):
    """Force reset all spiking layers"""
    for m in model.modules():
        if hasattr(m, "reset_mem"):
            m.reset_mem()

print("🚀 Initializing SNN U-Net...")
model = ProjectUNet(in_ch=5, n_classes=4).to(CONFIG['device'])
optimizer = optim.AdamW(model.parameters(), lr=CONFIG['lr'])
criterion = nn.CrossEntropyLoss()

# REMOVED SCALER (NO AMP)

best_mcc = -1.0
history = {'loss': [], 'mcc': []}

print(f"🔥 Starting SNN Training (BS={CONFIG['batch_size']}, Float32)...")

for epoch in range(CONFIG['epochs']):
    model.train()
    running_loss = 0

    pbar = tqdm(train_loader, desc=f"Ep {epoch+1}")
    for imgs, masks in pbar:
        imgs, masks = imgs.to(CONFIG['device']), masks.to(CONFIG['device']).long()

        # 1. Manual Reset (Critical)
        manual_reset(model)

        optimizer.zero_grad()

        # 2. Standard Forward/Backward (No Autocast)
        out = model(imgs)
        loss = criterion(out, masks)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        pbar.set_postfix({'loss': loss.item()})

    avg_loss = running_loss / len(train_loader)
    history['loss'].append(avg_loss)

    # Validation
    model.eval()
    preds, targets = [], []
    sample_vis = None

    with torch.no_grad():
        for i, (imgs, masks) in enumerate(val_loader):
            imgs = imgs.to(CONFIG['device'])

            manual_reset(model) # Reset before inference too!
            out = model(imgs)

            p = out.argmax(1).cpu()
            preds.append(p)
            targets.append(masks.cpu())

            if i == 0:
                sample_vis = (imgs[0].cpu().numpy(), masks[0].cpu().numpy(), p[0].numpy())

    mcc = matthews_corrcoef(torch.cat(targets).numpy().flatten(), torch.cat(preds).numpy().flatten())
    history['mcc'].append(mcc)

    print(f"Ep {epoch+1} | Loss: {avg_loss:.4f} | Val MCC: {mcc:.4f}")

    save_plot_and_sample(history, sample_vis, epoch+1)

    if mcc > best_mcc:
        best_mcc = mcc
        torch.save(model.state_dict(), f"{CONFIG['project_dir']}/best_SNN.pth")
        print("✅ Saved Best SNN Model!")

print(f"🏁 Training Complete. Best MCC: {best_mcc:.4f}")

🚀 Initializing SNN U-Net...
🔥 Starting SNN Training (BS=2, Float32)...


Ep 1: 100%|██████████| 10/10 [00:10<00:00,  1.05s/it, loss=1.16]


Ep 1 | Loss: 1.2273 | Val MCC: 0.2803
✅ Saved Best SNN Model!


Ep 2: 100%|██████████| 10/10 [00:08<00:00,  1.11it/s, loss=0.943]


Ep 2 | Loss: 1.0311 | Val MCC: 0.3838
✅ Saved Best SNN Model!


...

Ep 28 | Loss: 0.5470 | Val MCC: 0.4611


Ep 29: 100%|██████████| 10/10 [00:08<00:00,  1.14it/s, loss=0.401]


Ep 29 | Loss: 0.5434 | Val MCC: 0.4926


Ep 30: 100%|██████████| 10/10 [00:09<00:00,  1.06it/s, loss=0.557]


Ep 30 | Loss: 0.5280 | Val MCC: 0.4967
🏁 Training Complete. Best MCC: 0.5233
